# MongoDB (October 2018)
    

In [1]:
import re
import json
from pymongo import MongoClient

The file `trump.json` contains posts from http://www.twitter.com (tweets) over the course of an hour that have the key word "trump".
Each line in the file is a single JSON message that can be loaded with `json.loads()`.

We create a MongoDB database and initialize a collection in the database.

In [14]:
# Set up client
client = MongoClient()
db = client.db1
col = db.collection1

In [15]:
# Clear existing elements in collection
col.delete_many({})

In [16]:
# Read in trump tweets
with open('trump.json') as f:
    for line in f:
        col.insert_one(json.loads(line))

In [17]:
# check number of columns
col.count()

C:\Users\jbohm\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


95643

The following code queries the Twitter collection for the listed information:
- How many tweets include the word Russia?
- How many tweets came from one of the main continental US time zones? (These are listed as `"Central Time (US & Canada)"`, `"Pacific Time (US & Canada)"`, `"Eastern Time (US & Canada)"`, and `"Mountain Time (US & Canada)"`.)
- How often does each language occur? (To answer this, we will construct a dictionary with each language and it’s frequency count.)

In [20]:
# Display example of an entry in the collection for reference
col.find()[0]

{'_id': ObjectId('5bea09176fb853fb28a6bd5e'),
 'created_at': 'Fri May 26 19:18:30 +0000 2017',
 'id': 868184586844549121,
 'id_str': '868184586844549121',
 'text': 'RT @TruthFeedNews: LIBERAL SPIN: Democrats Are Saying That Losing in Montana is "A Win" https://t.co/t6QJFs0i4S #maga #trumptrain #trump',
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 15619384,
  'id_str': '15619384',
  'name': 'gab.ai/Meowma',
  'screen_name': 'Meowma',
  'location': 'Hollywood, California',
  'url': 'https://youtu.be/PkBxePSZ1ZI',
  'description': 'Liberated #Hollywood entertainment industry artist and producer. Exposing #scumbags...especially #illuminati, #warmongers and #zionists. #CorruptionDisruption',
  'protected': False,
  'verified': False,
  'followers_count': 5

In [21]:
# Count number of times the word Russia shows up in the collection
russia = col.find({"text":re.compile('Russia',re.IGNORECASE)})

In [22]:
# Display the number for tweets with the word 'Russia'
russia.count()

C:\Users\jbohm\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


5841

In [25]:
# Define the different time zones
zones = ["Central Time (US & Canada)", "Pacific Time (US & Canada)", "Eastern Time (US & Canada)", "Mountain Time (US & Canada)"]

In [31]:
# Find number of tweets from each time zone
timezone = col.find({"user.time_zone":{"$in":zones}})

In [32]:
# Display number of tweets from each time zone
timezone.count()

C:\Users\jbohm\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


33939

In [35]:
# Get distinct languages
langs = col.distinct("lang")

In [37]:
# Create dictionary of different languages and their frequencies, using dictionary comprehension
lang_dict = {l:col.find({"lang":l}).count() for l in langs}

C:\Users\jbohm\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


In [38]:
# Display the language dictionary
lang_dict

{'en': 84919,
 'fr': 1235,
 'es': 3027,
 'tr': 588,
 'nl': 212,
 'pt': 632,
 'und': 1653,
 'it': 423,
 'de': 635,
 'da': 35,
 'zh': 1,
 'ru': 29,
 'pl': 108,
 'no': 21,
 'eu': 6,
 'tl': 48,
 'in': 121,
 'cs': 13,
 'ht': 21,
 'ro': 21,
 'sl': 3,
 'sv': 55,
 'fi': 20,
 'ar': 30,
 'et': 23,
 'lt': 8,
 'hu': 6,
 'el': 10,
 'ja': 17,
 'lv': 2,
 'vi': 7,
 'fa': 5,
 'ko': 2,
 'ur': 1,
 'cy': 2,
 'th': 3,
 'is': 1,
 'iw': 1,
 'uk': 3}

The following code queries the Twitter collection from Problem 1 for the following information:
- What are the usernames of the 5 most popular (defined as having the most followers) tweeters? Don’t include repeats.
- Of the tweets containing at least 5 hashtags, sort the tweets by how early the 5th hashtag appears in the text. What is the earliest spot (character count) it appears?
- What are the coordinates of the tweet that came from the northernmost location? 

In [54]:
# Delete the Wall Street Journal from the collection to remove duplicates from the list of the 5 top most popular tweeters
col.delete_one({"user.name":"Wall Street Journal"})

In [55]:
# Get the most popular tweeters
popular = col.find().sort("user.followers_count",-1)

# Get the usernames of the 5 most popular tweeters
people = []
for pop in popular[:5]:
    people.append(pop["user"]["name"])

# Print the list of usernames of 5 most popular tweeters
people

['CNN', 'Reuters Top News', 'Fox News', 'Wall Street Journal', 'TIME']

In [79]:
# Find tags with at least 5 hashtags
tags = col.find({"entities.hashtags.4":{"$exists":True}})

# Sort the tags with at least 5 hashtags by how early the 5th hashtag appears in the tweet
sorted_tags = col.find({"entities.hashtags.4":{"$exists":True}}).sort('entities.hashtags.4.indices.0')
first = sorted_tags[0]
num = first["entities"]["hashtags"][4]["indices"][0]

# Display the hashtags for the hashtag with the earliest 5th hashtag
sorted_tags[0]["entities"]["hashtags"]

[{'text': 'FOREX', 'indices': [0, 6]},
 {'text': 'Follow', 'indices': [7, 14]},
 {'text': 'FX', 'indices': [15, 18]},
 {'text': 'RT', 'indices': [19, 22]},
 {'text': 'News', 'indices': [23, 28]}]

From the above output, we can see that the earliest position of a hashtag appearing the 5th hashtag of a tweet is 23.

In [84]:
coords = col.find({"coordinates.coordinates":{"$exists":True}}).sort("coordinates.coordinates.1")
print("Coordinates of northenmost tweet: ",coords[0]["coordinates"]["coordinates"])

Coordinates of northenmost tweet:  [28.03908658, -26.14493987]


We now clean the Twitter collection in the following ways:

- Get rid of the `"retweeted_status"` field in each tweet.
- Update every tweet from someone with at least 1000 followers to include a popular field whose value is True. Report the number of popular tweets.

In [87]:
# Get rid of the retweeted status field in each tweet
col.update_many({"retweeted_status":{"$exists":True}},{"$unset":{"retweeted_status":''}})

In [89]:
# Check that the retweeted field no longer exists
col.find({"retweeted_status":{"$exists":True}}).count()

C:\Users\jbohm\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


0

In [94]:
# Create new field to include popularity of a tweeter for their tweets
col.update_many({"user.followers_count":{"$gte":1000}},{"$set":{"popular":True}})
col.find({"popular":True}).count()

C:\Users\jbohm\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


32487

This implies there are actually 32489 tweets, because two tweets by Wall Street Journal that were within the top ten most popular tweets were deleted earlier in order to prevent repeats.